In [1]:
# importation des bibliothèques nécessaires 
import pandas as pd 
import numpy as np 
import time 
import tensorflow as tf 
from sklearn.preprocessing import robust_scale
from sklearn.cross_validation import train_test_split
import numpy as np

# lecteur des données pour l'entrainement du modèle 
df=pd.read_csv('donnee/data.csv',header=0,index_col=0)

# céation de numpy array qui sera utilisé pour l'entrainement et le test du modèle 
X= df.iloc[:,4:-1]
names = X.columns
X=X.values
# création des données pour les deux classes 
Y= np.random.choice(2, X.shape[0]).reshape(X.shape[0],1)
for i in range(len(Y)):
    if Y[i]==0:
        X[i,:]=-X[i,:]
# standardisation des données ( rosbust aux outliars)
X=robust_scale(X)
# division des données en entrainement et test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=0,test_size=0.25)

# nombres des variable et nombre des classe 
num_features = X_train.shape[1]
num_classes =  Y_train.shape[1]
X_train

/home/helmi/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([[-0.42706626,  0.13985797,  0.27001278, ..., -3.30653266,
        -0.39347982,  0.51542206],
       [ 0.13941895, -0.52060659,  0.66711661, ..., -1.77889447,
        -0.44500362,  0.93784994],
       [ 0.12687574,  1.01325975,  0.15535855, ...,  1.10552764,
        -0.2864642 , -0.19288726],
       ...,
       [-0.60769195,  0.59612979,  1.37291676, ..., -0.        ,
        -0.63054687, -0.00784261],
       [-0.10665123, -0.02384519,  1.22197312, ...,  0.2361809 ,
        -0.56187814, -1.01922979],
       [-0.00776943,  1.4644058 ,  2.23509447, ..., -0.        ,
        -0.27282043, -0.09883803]])

In [74]:
# création des placeholders les trous qui seront remplis par les données d'entrainemet
x = tf.placeholder( shape=[None, num_features] ,dtype=tf.float32 )
y = tf.placeholder(shape=[None, num_classes] , dtype=tf.float32)

#les paramétres du modèle 
W = tf.Variable(tf.zeros([num_features,num_classes]),dtype=tf.float32)
B = tf.Variable(tf.zeros([num_classes]),dtype=tf.float32)

# la fonction du regression lostique 
pY =tf.nn.sigmoid(tf.matmul(x, W) + B)
# fonction cout à minimiser 
cost_fn= tf.reduce_mean(tf.reduce_sum((-y * tf.log(pY)) - ((1 - y) * tf.log(1 - pY)), reduction_indices=[1]))# You could also put it in a more explicit way

# optimizer adam plus rapide que Gradient descent sa pas change selon la valeur de dérivé 
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)



In [85]:
# les hyper-paramètres 
learning_rate=0.01
match_number=X.shape[0]
features_number=X.shape[1]
batch_size=100
number_batchs=int(match_number/batch_size)
n_epochs=1000

In [54]:
# choisir un batch de façon aléatoire pour l'entrainement du modèle en donnant la taille de batch 
def batch_choice(x,y, batch_size):
    indexs=np.random.choice(a=x.shape[0],size=batch_size,replace=False)
    x=x[indexs,:]
    y=y[indexs,:]
    return x.astype(float),y.astype(float)



In [82]:
# fonction de prédection 
def pred(x):
    l=[]
    for i in list(x) :
        if i > 0.5 :
            l.append(1)
        else:
            l.append(0)
    return l

In [6]:
# fonction score 
def score (x,y):
    l=[]
    for i,j in zip (x,y):
        if i==j:
            l.append(1)
        else:
            l.append(0)
    return (np.mean(l))
    

In [86]:
# entrainement du modèle 
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

costs=[]
scores=[]
for i in range(n_epochs):
    avg_cost = 0.
        
    for i in range(number_batchs):
        x_train , y_train = batch_choice (X_train, Y_train , batch_size)
        model = sess.run(optimizer, feed_dict={x:x_train,y:y_train})
        c = sess.run(cost_fn, feed_dict={x:x_train,y:y_train})
        avg_cost+=c/batch_size
           
    costs.append(avg_cost)
    p=sess.run(pY , feed_dict={x:X_test})
    prediction=pred(p)
    scores.append(score(prediction,Y_test))
                




    
   



In [93]:
p=sess.run(pY , feed_dict={x:X_test})
